In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from vaastav.understat import *
from vaastav.getters import *
from config import *

In [3]:
fixtures = get_fixtures_data()

In [4]:
df_fixtures = pd.DataFrame(fixtures).dropna()

In [5]:
def team_id_to_name(current_teams, team_id):
    return current_teams[team_id - 1]

In [6]:
df_fixtures["Squad H"] = df_fixtures["team_h"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["Squad A"] = df_fixtures["team_a"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["date"] = df_fixtures["kickoff_time"].str[:10]

In [7]:
# df_fixtures

In [8]:
epl_data = get_epl_data()
epl_data = epl_data[0]

In [9]:
def team_understat_to_fpl(team_name):
    if team_name == "Wolverhampton Wanderers":
        return "Wolves"
    if team_name == "Newcastle United":
        return "Newcastle Utd"
    if team_name == "Manchester United":
        return "Manchester Utd"
    if team_name == "Nottingham Forest":
        return "Nott'ham Forest"
    return team_name

In [10]:
df = pd.DataFrame(columns=["Team", "xG", "xGA", "Date"])

for k in epl_data.keys():
    title = team_understat_to_fpl(epl_data[k]['title'])
    for h in epl_data[k]['history']:
        xG = h["xG"]
        xGA = h["xGA"]
        date = h["date"][:10]
        
        new_row = {
                    "Team": title,
                    "xG": xG,
                    "xGA": xGA,
                    "Date": date
                  }
        df = df.append(new_row, ignore_index=True)

In [11]:
def assign_opponent(x):
    r1 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad H"] == x["Team"]) ]
    r2 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad A"] == x["Team"]) ]
    if r1.size > 0:
        return r1["Squad A"].item()
    if r2.size > 0:
        return r2["Squad H"].item()
    return None

In [12]:
# df_fixtures

In [13]:
# df_fixtures["Squad A"].unique()

In [14]:
# df["Team"].unique()

In [15]:
for i, row in df.iterrows():
    df.loc[i, "Opponent"] = assign_opponent(row)

In [16]:
summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()
summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()
# Add a summary row for each unique 'Team'
# summary_df.append({'Team': 'Summary', 'xG': summary_df['xG'].sum(), 'xGA': summary_df['xGA'].sum()}, ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15764\2432573045.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df = df.groupby('Team', as_index=False)['xG', 'xGA'].sum()
C:\Users\user\AppData\Local\Temp\ipykernel_15764\2432573045.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_df_per_90 = df.groupby('Team', as_index=False)['xG', 'xGA'].mean()


In [17]:
summary_df["Opponent"] = "Summary"
summary_df_per_90["Opponent"] = "Summary per 90"

In [18]:
# summary_df

In [19]:
df = df.append(summary_df)
df = df.append(summary_df_per_90)
df = df.reset_index()

In [20]:
df.tail(40)

,index,Team,xG,xGA,Date,Opponent
52,0,Arsenal,6.279430,2.885282,NaN,Summary
53,1,Aston Villa,4.729360,5.043545,NaN,Summary
54,2,Bournemouth,3.380532,6.999030,NaN,Summary
55,3,Brentford,6.324451,2.913669,NaN,Summary
56,4,Brighton,8.897400,7.526850,NaN,Summary
57,5,Burnley,0.311032,2.400740,NaN,Summary
58,6,Chelsea,6.955930,3.478021,NaN,Summary
59,7,Crystal Palace,4.405930,3.576645,NaN,Summary
60,8,Everton,5.209395,6.227080,NaN,Summary
61,9,Fulham,2.831156,9.289770,NaN,Summary


In [21]:
# assigning weighted
for i, row in df[~df["Date"].isna()].iterrows():
    # print(df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")])
    df.loc[i, "Weighted_xG"] = row["xG"] / df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")]["xGA"].item()
    df.loc[i, "Weighted_xGA"] = row["xGA"] / df[(df["Team"] == row["Opponent"]) & (df["Opponent"] == "Summary per 90")]["xG"].item()

In [22]:
df.head(50)

,index,Team,xG,xGA,Date,Opponent,Weighted_xG,Weighted_xGA
0,0,Aston Villa,1.486000,4.322080,2023-08-12,Newcastle Utd,1.096161,1.887687
1,1,Aston Villa,3.243360,0.721465,2023-08-20,Everton,1.562543,0.415479
2,2,Everton,2.590010,1.581440,2023-08-12,Fulham,0.836407,1.675754
3,3,Everton,0.721465,3.243360,2023-08-20,Aston Villa,0.286094,1.371585
4,4,Everton,1.897920,1.402280,2023-08-26,Wolves,0.810568,0.723724
5,5,Bournemouth,1.510250,1.483400,2023-08-12,West Ham,1.008243,0.723793
6,6,Bournemouth,1.386700,3.254480,2023-08-19,Liverpool,0.835543,1.429152
7,7,Bournemouth,0.483582,2.261150,2023-08-26,Tottenham,0.299916,1.084771
8,8,Crystal Palace,2.276160,0.422704,2023-08-12,Sheffield United,1.247905,0.868205
9,9,Crystal Palace,1.069160,2.205930,2023-08-21,Arsenal,1.111670,1.053884


In [23]:
summary_weighted_df = df.groupby('Team', as_index=False)['Weighted_xG', "Weighted_xGA"].mean()

C:\Users\user\AppData\Local\Temp\ipykernel_15764\1221973474.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  summary_weighted_df = df.groupby('Team', as_index=False)['Weighted_xG', "Weighted_xGA"].mean()


In [27]:
summary_weighted_df.sort_values(by=["Weighted_xG"], ascending=False)

,Team,Weighted_xG,Weighted_xGA
13,Manchester Utd,1.471062,0.993473
1,Aston Villa,1.329352,1.151583
14,Newcastle Utd,1.309541,0.652121
17,Tottenham,1.308055,0.773216
10,Liverpool,1.258123,1.032051
0,Arsenal,1.155114,0.835786
7,Crystal Palace,1.150537,0.790593
4,Brighton,1.129232,1.477338
18,West Ham,1.096564,0.832546
5,Burnley,1.094825,1.324174


In [25]:
# summary_weighted_df["Weighted_xG"].sum() / 20